In [ ]:
import pandas as pd

# Master algo

In [ ]:
def Master:
    """
    Master algorithm managing the collaboration between the local data sites
    """
    
    def __init__(self, collabs=None, Xs=None, Ys=None, *args, **kwargs):
        """
        collabs: list(Collaborator)
            List of instances of type "Collaborator".
        X: list(pandas.DataFrame)
            List of dataframes with same size than collabs.
            The view of each collaborators, with acces to possibly different
            set of individuals (identified by the index column).
        Y (optional): list(pandas.DataFrame)
            List of dataframes, one for each collaborator. With the indices and the labels.
            
        notes:
        02/02 14:16 - Xs and Ys are not necessary parameters, as we can get them from each collab.
                      Consider removing them.
        """
        if collabs:
            self.collabs, self.P = collabs, len(collabs)
            
        if X:
            self.X = X
            
        if Y:
            self.Y = Y
            
    def launch_collab(self):
        """
        Proceed to the collaboration.
        
        """
        
        # each collaborator fits its parameters in the local step
        for collab in self.collabs:
            collab.local_step()
            
        while True:
             
    

# collaborator

In [ ]:
class Collaborator():
    """
    An abstract class representing a collaborator.
    Collaborators can be of different types (running with different algorithms)
    e.g. Gtms, gaussian mixture models, other mixture models, or any kind of probabilistic model.
    """
    
    def __init__(self, X, Y=None, *args, **kwargs):
        """
        Parameters:
            X: pandas.DataFrame
                Dataframe with the data we have at our disposal
            
            
        """

# tests

In [14]:
class A:
    def __init__(self, a=0):
        self.a = a
        
    def double_a(self):
        try:
            return 2*self.a
        except:
            print("no attribute a")
        
class B(A):
    def __init__(self, a=0, b=0):
        self.a=a
        self.b=b

In [15]:
b = B()

In [16]:
b.double_a()

0